In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.preprocessing import MinMaxScaler

%load_ext tensorboard

train_df = pd.read_csv('data/features.csv')

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")


In [2]:
train_x = train_df.drop(columns=['precio'])
train_y = train_df.precio

In [3]:
import keras as keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (0,1))

train_x.antiguedad = scaler.fit_transform(np.array(train_x.antiguedad).reshape(-1, 1))
train_x.metroscubiertos = scaler.fit_transform(np.array(train_x.metroscubiertos).reshape(-1, 1))
train_x.metrostotales = scaler.fit_transform(np.array(train_x.metrostotales).reshape(-1, 1))
#inverse = scaler.inverse_transform(normalized)

early_stopping_monitor = EarlyStopping(patience=10)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

#get number of columns in training data
n_cols = train_x.shape[0]

wide_model = Sequential()

wide_model.add(Dense(10000, activation = 'relu', input_shape = (43,)))
wide_model.add(Dense(1))

wide_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
training_history = wide_model.fit(train_x, train_y, validation_split=0.2, epochs=1000, callbacks=[tensorboard_callback])
 

Using TensorFlow backend.


Train on 164176 samples, validate on 41044 samples
Epoch 1/1000
164176/164176 [==============================] - 26s 156us/step - loss: 6647014399558.5059 - mae: 1753711.7500 - val_loss: 3404545369212.8438 - val_mae: 1228316.7500
Epoch 2/1000
164176/164176 [==============================] - 25s 153us/step - loss: 3053293452547.1685 - mae: 1260207.7500 - val_loss: 2907511004624.3477 - val_mae: 1242380.0000
Epoch 3/1000
164176/164176 [==============================] - 25s 153us/step - loss: 2724394484722.7275 - mae: 1200425.2500 - val_loss: 2596275457161.4180 - val_mae: 1156639.1250
Epoch 4/1000
164176/164176 [==============================] - 25s 154us/step - loss: 2433930482474.7373 - mae: 1114338.1250 - val_loss: 2322893232634.3613 - val_mae: 1067823.0000
Epoch 5/1000
164176/164176 [==============================] - 25s 154us/step - loss: 2194025620256.7578 - mae: 1031623.7500 - val_loss: 2113710577793.5344 - val_mae: 991992.6250
Epoch 6/1000
164176/164176 [===========================

In [4]:
deep_model = Sequential()

# La idea es que los modelos mas profundos pueden abstraerse mejor de los datos con lo que fueron entrenados
deep_model.add(Dense(50, activation = 'relu', input_shape = (43,)))
deep_model.add(Dense(100, activation = 'relu'))
deep_model.add(Dense(150, activation = 'relu'))
deep_model.add(Dense(200, activation = 'relu'))
deep_model.add(Dense(150, activation = 'relu'))
deep_model.add(Dense(100, activation = 'relu'))
deep_model.add(Dense(50, activation = 'relu'))
deep_model.add(Dense(1))

deep_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
training_history = deep_model.fit(train_x, train_y, validation_split=0.2, epochs=1000, callbacks=[tensorboard_callback])

Train on 164176 samples, validate on 41044 samples
Epoch 1/1000
164176/164176 [==============================] - 17s 103us/step - loss: 1792935120732.7349 - mae: 847945.7500 - val_loss: 1699616154942.0479 - val_mae: 877074.6875
Epoch 2/1000
164176/164176 [==============================] - 17s 101us/step - loss: 1539043235409.0837 - mae: 769724.9375 - val_loss: 1573141770666.7249 - val_mae: 740336.5625
Epoch 3/1000
164176/164176 [==============================] - 17s 101us/step - loss: 1500462316629.1255 - mae: 759182.5000 - val_loss: 1493326597813.0786 - val_mae: 769343.8125
Epoch 4/1000
164176/164176 [==============================] - 16s 100us/step - loss: 1472184019390.8835 - mae: 751163.7500 - val_loss: 1450157822638.1929 - val_mae: 727758.7500
Epoch 5/1000
164176/164176 [==============================] - 17s 101us/step - loss: 1439320011491.6831 - mae: 742697.0625 - val_loss: 1416405030302.2505 - val_mae: 723468.0000
Epoch 6/1000
164176/164176 [==============================] - 17

In [8]:
test_df = pd.read_csv('data/testModified.csv')
test_df.antiguedad = scaler.fit_transform(np.array(test_df.antiguedad).reshape(-1, 1))
test_df.metroscubiertos = scaler.fit_transform(np.array(test_df.metroscubiertos).reshape(-1, 1))
test_df.metrostotales = scaler.fit_transform(np.array(test_df.metrostotales).reshape(-1, 1))

#deep_model_prediction = deep_model.predict(test_df.drop(columns=['id']))
res = pd.DataFrame(deep_model, index=test_df.id, columns=['precio'])
res = res.rename({'precio':'target'}, axis=1)
res.to_csv("predictions/deep_model_prediction.csv", header = True)

In [9]:
deep_model

array([[6756809.  ],
       [ 497338.22],
       [2102467.  ],
       ...,
       [1133866.1 ],
       [1253792.1 ],
       [2763418.2 ]], dtype=float32)

In [1]:
test_df = pd.read_csv('data/testModified.csv')
test_df.antiguedad = scaler.fit_transform(np.array(test_df.antiguedad).reshape(-1, 1))
test_df.metroscubiertos = scaler.fit_transform(np.array(test_df.metroscubiertos).reshape(-1, 1))
test_df.metrostotales = scaler.fit_transform(np.array(test_df.metrostotales).reshape(-1, 1))

wide_model_prediction = wide_model.predict(test_df.drop(columns=['id']))
res = pd.DataFrame(wide_model, index=test_df.id, columns=['precio'])
res = res.rename({'precio':'target'}, axis=1)
res.to_csv("predictions/wide_model_prediction.csv", header = True)

NameError: name 'pd' is not defined

In [2]:
print(wide_model)

NameError: name 'wide_model' is not defined